# Chat Engines with Memory

Chat engines enable conversational interactions with your data, maintaining context across multiple turns. This is essential for building chatbots and conversational assistants.

## Learning Objectives

By the end of this notebook, you will:
1. Understand the difference between query engines and chat engines
2. Implement different chat modes
3. Customize conversation memory
4. Build a complete chatbot with RAG
5. Handle conversation state and context

---

## Query Engine vs Chat Engine

| Feature | Query Engine | Chat Engine |
|---------|--------------|-------------|
| Memory | None | Maintains conversation history |
| Context | Single query | Multi-turn context |
| Use Case | One-off questions | Conversations |
| Pronouns | Can't resolve "it", "that" | Understands references |

In [ ]:
# Setup
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
)
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Configure
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

print("✓ Setup complete!")

In [ ]:
# Load and index documents
documents = SimpleDirectoryReader("../data/sample_docs").load_data()
index = VectorStoreIndex.from_documents(documents, show_progress=True)

print(f"\n✓ Index ready!")

## 1. Basic Chat Engine

The simplest way to create a chat engine from an index:

In [ ]:
# Create a basic chat engine
chat_engine = index.as_chat_engine(
    chat_mode="condense_question",  # Reformulates follow-ups
    verbose=True,
)

print("✓ Chat engine ready!")

In [ ]:
# Start a conversation
print("=" * 60)
print("CONVERSATION START")
print("=" * 60)

# First message
response1 = chat_engine.chat("What is machine learning?")
print(f"\nUser: What is machine learning?")
print(f"Assistant: {response1}")

In [ ]:
# Follow-up that references previous context
response2 = chat_engine.chat("What are its main types?")
print(f"\nUser: What are its main types?")
print(f"Assistant: {response2}")

In [ ]:
# Another follow-up
response3 = chat_engine.chat("Can you give an example of the first type?")
print(f"\nUser: Can you give an example of the first type?")
print(f"Assistant: {response3}")

In [ ]:
# View conversation history
print("\n" + "=" * 60)
print("CONVERSATION HISTORY")
print("=" * 60)

for msg in chat_engine.chat_history:
    role = msg.role.value.upper()
    print(f"\n{role}: {str(msg.content)[:200]}...")

In [ ]:
# Reset conversation
chat_engine.reset()
print("✓ Conversation reset!")

## 2. Chat Modes

LlamaIndex offers different chat modes for different use cases:

| Mode | Description | Best For |
|------|-------------|----------|
| `condense_question` | Reformulates follow-ups into standalone queries | General RAG chat |
| `context` | Retrieves context for every message | Simple Q&A |
| `condense_plus_context` | Combines both approaches | Complex conversations |
| `simple` | Direct LLM chat (no retrieval) | General chat |
| `react` | Agent-style with reasoning | Tool use |

In [ ]:
# Compare different chat modes
def test_chat_mode(mode, messages):
    """Test a chat mode with a series of messages."""
    engine = index.as_chat_engine(chat_mode=mode)
    
    print(f"\n{'='*60}")
    print(f"Chat Mode: {mode}")
    print("=" * 60)
    
    for msg in messages:
        response = engine.chat(msg)
        print(f"\nUser: {msg}")
        print(f"Assistant: {str(response)[:200]}...")
    
    return engine

In [ ]:
# Test messages
test_messages = [
    "What is Python?",
    "What are its main uses?",
    "How does it compare to other languages?",
]

# Test condense_question mode
engine1 = test_chat_mode("condense_question", test_messages)

In [ ]:
# Test context mode
engine2 = test_chat_mode("context", test_messages)

## 3. Custom Memory Configuration

Control how conversation history is managed:

In [ ]:
# Create custom memory buffer
memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,  # Maximum tokens to keep in memory
)

# Create chat engine with custom memory
custom_chat_engine = index.as_chat_engine(
    chat_mode="condense_question",
    memory=memory,
    verbose=True,
)

print("✓ Chat engine with custom memory ready!")

In [ ]:
# Long conversation to test memory
conversation = [
    "Tell me about artificial intelligence.",
    "What about machine learning specifically?",
    "How does deep learning fit into this?",
    "What are some practical applications?",
    "Can you summarize what we've discussed?",
]

print("Testing long conversation with memory...\n")

for msg in conversation:
    response = custom_chat_engine.chat(msg)
    print(f"User: {msg}")
    print(f"Assistant: {str(response)[:150]}...\n")

In [ ]:
# Check memory status
print(f"Messages in memory: {len(memory.get_all())}")
print(f"\nMemory contents:")
for i, msg in enumerate(memory.get_all()):
    print(f"  {i+1}. {msg.role}: {str(msg.content)[:50]}...")

## 4. Streaming Chat

Stream chat responses for better user experience:

In [ ]:
# Create streaming chat engine
streaming_chat = index.as_chat_engine(
    chat_mode="condense_question",
    streaming=True,
)

print("✓ Streaming chat engine ready!")

In [ ]:
# Stream a response
query = "Explain the key concepts of object-oriented programming."

print(f"User: {query}\n")
print("Assistant: ", end="")

streaming_response = streaming_chat.stream_chat(query)

for token in streaming_response.response_gen:
    print(token, end="", flush=True)

print("\n")

## 5. Custom System Prompt

Customize the chat assistant's personality and behavior:

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole

# Custom system prompt
system_prompt = """
You are a helpful AI programming tutor specializing in Python and AI.
Your responses should be:
- Educational and encouraging
- Include code examples when relevant
- Explain concepts step by step
- Ask clarifying questions if the user's question is unclear

If asked about topics outside your knowledge base, politely redirect
the conversation back to programming and AI topics.
"""

# Create chat engine with custom system prompt
tutor_chat = index.as_chat_engine(
    chat_mode="condense_question",
    system_prompt=system_prompt,
    verbose=False,
)

print("✓ Tutor chat engine ready!")

In [ ]:
# Test the tutor
tutor_questions = [
    "I'm new to programming. What should I learn first?",
    "Can you show me how to write a simple function?",
    "What's the difference between a list and a tuple?",
]

print("Programming Tutor Session")
print("=" * 60)

for q in tutor_questions:
    response = tutor_chat.chat(q)
    print(f"\nStudent: {q}")
    print(f"\nTutor: {response}\n")
    print("-" * 60)

## 6. Building a Complete Chatbot

Let's create a reusable chatbot class:

In [ ]:
from typing import Optional, List, Generator
from dataclasses import dataclass
from datetime import datetime

@dataclass
class ChatMessage:
    role: str
    content: str
    timestamp: datetime

class RAGChatbot:
    """A complete RAG-powered chatbot with conversation management."""
    
    def __init__(
        self,
        index: VectorStoreIndex,
        system_prompt: Optional[str] = None,
        chat_mode: str = "condense_question",
        memory_token_limit: int = 3000,
    ):
        self.index = index
        self.system_prompt = system_prompt
        self.chat_mode = chat_mode
        self.memory_token_limit = memory_token_limit
        
        self._init_chat_engine()
        self.conversation_log: List[ChatMessage] = []
    
    def _init_chat_engine(self):
        """Initialize the chat engine."""
        memory = ChatMemoryBuffer.from_defaults(
            token_limit=self.memory_token_limit
        )
        
        kwargs = {
            "chat_mode": self.chat_mode,
            "memory": memory,
        }
        
        if self.system_prompt:
            kwargs["system_prompt"] = self.system_prompt
        
        self.chat_engine = self.index.as_chat_engine(**kwargs)
    
    def chat(self, message: str) -> str:
        """Send a message and get a response."""
        # Log user message
        self.conversation_log.append(ChatMessage(
            role="user",
            content=message,
            timestamp=datetime.now(),
        ))
        
        # Get response
        response = self.chat_engine.chat(message)
        response_text = str(response)
        
        # Log assistant response
        self.conversation_log.append(ChatMessage(
            role="assistant",
            content=response_text,
            timestamp=datetime.now(),
        ))
        
        return response_text
    
    def stream_chat(self, message: str) -> Generator[str, None, None]:
        """Stream a response token by token."""
        # Create streaming engine temporarily
        streaming_engine = self.index.as_chat_engine(
            chat_mode=self.chat_mode,
            memory=self.chat_engine.memory,
            system_prompt=self.system_prompt,
            streaming=True,
        )
        
        self.conversation_log.append(ChatMessage(
            role="user",
            content=message,
            timestamp=datetime.now(),
        ))
        
        response = streaming_engine.stream_chat(message)
        full_response = ""
        
        for token in response.response_gen:
            full_response += token
            yield token
        
        self.conversation_log.append(ChatMessage(
            role="assistant",
            content=full_response,
            timestamp=datetime.now(),
        ))
    
    def reset(self):
        """Reset the conversation."""
        self.chat_engine.reset()
        self.conversation_log.clear()
    
    def get_history(self) -> List[dict]:
        """Get conversation history."""
        return [
            {
                "role": msg.role,
                "content": msg.content,
                "timestamp": msg.timestamp.isoformat(),
            }
            for msg in self.conversation_log
        ]
    
    def export_conversation(self) -> str:
        """Export conversation as formatted text."""
        lines = ["=== Conversation Export ===", ""]
        
        for msg in self.conversation_log:
            role = msg.role.upper()
            time = msg.timestamp.strftime("%H:%M:%S")
            lines.append(f"[{time}] {role}:")
            lines.append(msg.content)
            lines.append("")
        
        return "\n".join(lines)

print("✓ RAGChatbot class defined!")

In [ ]:
# Create and use the chatbot
chatbot = RAGChatbot(
    index=index,
    system_prompt="You are a helpful AI assistant that explains technical concepts clearly.",
    chat_mode="condense_question",
    memory_token_limit=4000,
)

print("✓ Chatbot initialized!\n")

# Have a conversation
questions = [
    "What is artificial intelligence?",
    "How is it different from machine learning?",
    "What role does Python play in this field?",
]

for q in questions:
    print(f"You: {q}")
    response = chatbot.chat(q)
    print(f"\nBot: {response[:300]}...\n")
    print("-" * 60)

In [ ]:
# Export conversation
print(chatbot.export_conversation())

## 7. Handling Context Window Limits

When conversations get long, you need to manage the context window:

In [ ]:
from llama_index.core.memory import (
    ChatMemoryBuffer,
    VectorMemory,
)

# Vector-based memory for long conversations
# Retrieves relevant past messages instead of keeping all
vector_memory = VectorMemory.from_defaults(
    vector_store=None,  # Uses in-memory store
    embed_model=Settings.embed_model,
    retriever_kwargs={"similarity_top_k": 3},
)

# Create chat engine with vector memory
long_chat = index.as_chat_engine(
    chat_mode="condense_question",
    memory=vector_memory,
)

print("✓ Chat engine with vector memory ready!")
print("(Retrieves relevant past messages instead of keeping all)")

## 8. Summary

You've learned how to build conversational interfaces with LlamaIndex:

### Key Takeaways

| Feature | Description |
|---------|-------------|
| **Chat Modes** | Different strategies for handling conversation context |
| **Memory** | Maintains conversation history across turns |
| **Streaming** | Better UX with token-by-token output |
| **System Prompt** | Customize assistant personality and behavior |
| **Vector Memory** | Efficient handling of long conversations |

### Best Practices

1. **Use `condense_question`** for most RAG chat applications
2. **Set appropriate memory limits** based on your LLM's context window
3. **Always stream** for user-facing applications
4. **Customize system prompts** for your use case
5. **Log conversations** for debugging and improvement

### Next Steps

In the Advanced section, we'll explore:
- Building agents with tools
- Complex workflows
- Custom components

---

## Exercises

1. **Custom persona**: Create a chat engine with a unique personality

2. **Conversation analytics**: Add metrics tracking to the chatbot class

3. **Multi-turn evaluation**: Test how well context is maintained over many turns

4. **Hybrid memory**: Combine buffer and vector memory strategies

In [ ]:
# Exercise space
# Build your custom chatbot here!